<a href="https://colab.research.google.com/github/NeuralClassifier/ReusabilityRate_ML/blob/main/CMPUT_663_ReuseRate_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 

In [5]:
!git clone https://github.com/NeuralClassifier/ReusabilityRate_ML

Cloning into 'ReusabilityRate_ML'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [8]:
pd.read_csv('/content/ReusabilityRate_ML/packages.csv',error_bad_lines=False)

,;Project;LongName;Name;AD;CD;TCD;CLOC;TCLOC;PDA;TPDA;LLOC;LOC;TLOC;NG;TLLOC;TNA;TNG;TNM;TNOS;TNPM;NCL;ReuseRate
0,0;ch.qos.logback/logback-classic;<root_package...
1,1;ch.qos.logback/logback-classic;ch;ch;0.0;0.0...
2,2;ch.qos.logback/logback-classic;unnamed packa...
3,3;ch.qos.logback/logback-classic;ch.qos;qos;0....
4,4;ch.qos.logback/logback-classic;ch.qos.logbac...
...,...
2188,2188;xerces/xercesImpl;org.apache.xerces.impl....
2189,2189;xerces/xercesImpl;org.apache.xerces.impl....
2190,2190;xerces/xercesImpl;org.apache.xerces.impl....
2191,2191;xerces/xercesImpl;org.apache.xerces.impl....
